In [1]:
import torch
import yaml
from model import HFModel


with open('configs/ants.yaml', 'r') as f:
        cfg = yaml.safe_load(f)
model = HFModel(model_name=cfg['model_name'], num_classes=cfg['num_classes'], predict_per_item=cfg['predict_per_item'])

/mnt/aperto/anaconda/envs/feral/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda:2')
model = model.to(device)

x = torch.rand(4, 8, 3, 128, 128, device=device)
model(x)

tensor([[-0.4409,  0.7933,  0.8316],
        [-0.5529,  0.7082,  0.8037],
        [-0.4003,  0.6543,  0.7291],
        [-0.4463,  0.6978,  0.7601],
        [-0.6452,  0.5759,  0.5980],
        [-0.4590,  0.6769,  0.6738],
        [-0.5591,  0.4920,  0.7761],
        [-0.4273,  0.4845,  0.6166],
        [-0.3632,  0.6281,  0.7941],
        [-0.4657,  0.5509,  0.7245],
        [-0.4006,  0.6374,  0.7626],
        [-0.4391,  0.5453,  0.8259],
        [-0.4009,  0.6585,  0.7256],
        [-0.4320,  0.7479,  1.0331],
        [-0.3193,  0.8255,  0.9509],
        [-0.3624,  0.5236,  0.7024],
        [-0.1114, -0.0171, -0.0689],
        [-0.2205, -0.0874, -0.1469],
        [-0.1114, -0.1832, -0.2046],
        [-0.1241, -0.1262, -0.1425],
        [-0.3375, -0.2134, -0.2999],
        [-0.1197, -0.1408, -0.2865],
        [-0.2664, -0.3375, -0.1388],
        [-0.1754, -0.2948, -0.3379],
        [-0.0944, -0.2247, -0.1270],
        [-0.1765, -0.2509, -0.2232],
        [-0.0756, -0.1913, -0.1406],
 

In [3]:
import torch.nn.functional as F
import torch.nn as nn

# Save the original function
original_sdp_attention = F.scaled_dot_product_attention
original_forward = nn.MultiheadAttention.forward
original_mha_fwd = F.multi_head_attention_forward

# Define the wrapper
def patched_sdp_attention(query, key, value, *args, **kwargs):
    print("scaled_dot_product_attention called! Query shape:", query.shape)
    return original_sdp_attention(query, key, value, *args, **kwargs)

F.scaled_dot_product_attention = patched_sdp_attention

def debug_forward(self, *args, **kwargs):
    print("MultiheadAttention forward called")
    return original_forward(self, *args, **kwargs)

nn.MultiheadAttention.forward = debug_forward

def debug_mha_fwd(*args, **kwargs):
    print("multi_head_attention_forward")
    return original_mha_fwd(*args, **kwargs)

F.multi_head_attention_forward = debug_mha_fwd

In [7]:
x = torch.rand(4, 8, 3, 128, 128, device=device)
with torch.backends.cuda(enable_math=False, enable_mem_efficient=False), torch.amp.autocast(dtype=torch.bfloat16, device_type="cuda"):
    model(x)

In [20]:
model

HFModel(
  (model): SmolVLMVisionTransformer(
    (embeddings): SmolVLMVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
      (position_embedding): Embedding(1024, 768)
    )
    (encoder): SmolVLMEncoder(
      (layers): ModuleList(
        (0-11): 12 x SmolVLMEncoderLayer(
          (self_attn): SmolVLMVisionAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): SmolVLMVisionMLP(
            (activation_fn): PytorchGELUTanh()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias

In [7]:
# torch.backends.cuda.math_sdp_enabled()
torch.backends.cuda.flash_sdp_enabled()

True

In [5]:
torch.backends.mha.get_fastpath_enabled()

True

In [7]:
torch.backends.cuda.mem_efficient_sdp_enabled(), torch.backends.cuda.math_sdp_enabled(), torch.backends.cuda.flash_sdp_enabled()

(True, True, True)

In [8]:
torch.backends.cuda.can_use_flash_attention()

TypeError: can_use_flash_attention() missing 1 required positional argument: 'params'